In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join("..", "src"))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from backtesting import Backtest
from strategies.bollinger_bands_breakout import BollingerBandsBreakout

/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [5]:
import yfinance as yf

nvda = yf.Ticker("NVDA").history(start="2023-01-01", end="2024-12-31", actions=False)

In [6]:
bt = Backtest(nvda, BollingerBandsBreakout,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
bt.plot()

GridPlot(id='p1510', ...)